---
title: 💰 Trillion Dollar Words in Julia
subtitle: JuliaCon 2024
author: Patrick Altmeyer
format:
  julia-revealjs: 
    scrollable: true
    title-slide-attributes:
      data-background-image: ../www/qr.png
    self-contained: true
date: 2024-07-11
date-format: full
bibliography: biblio.bib
execute:
  freeze: auto
  eval: true
  echo: true
  output: false
---

```{julia}
#| echo: false
projectdir = splitpath(pwd()) |>
    ss -> joinpath(ss[1:findall([s == "pat-alt.github.io" for s in ss])[1]]...) 
cd(projectdir)

include("$projectdir/content/talks/posts/2024-juliacon/trillion-dollar-words/setup.jl")
```


## [TrillionDollarWords.jl](https://github.com/pat-alt/TrillionDollarWords.jl)

[![Stable](https://img.shields.io/badge/docs-stable-blue.svg)](https://pat-alt.github.io/TrillionDollarWords.jl/stable/)
[![Dev](https://img.shields.io/badge/docs-dev-blue.svg)](https://pat-alt.github.io/TrillionDollarWords.jl/dev/)
[![Build Status](https://github.com/pat-alt/TrillionDollarWords.jl/actions/workflows/CI.yml/badge.svg?branch=main)](https://github.com/pat-alt/TrillionDollarWords.jl/actions/workflows/CI.yml?query=branch%3Amain)
[![Coverage](https://codecov.io/gh/pat-alt/TrillionDollarWords.jl/branch/main/graph/badge.svg)](https://codecov.io/gh/pat-alt/TrillionDollarWords.jl)
[![Code Style: Blue](https://img.shields.io/badge/code%20style-blue-4495d1.svg)](https://github.com/invenia/BlueStyle)

A light-weight package providing Julia users easy access to the Trillion Dollar Words dataset and model [@shah2023trillion].

::: {.callout-note}

## Disclaimer  

Please note that I am not the author of the Trillion Dollar Words paper nor am I affiliated with the authors. The package was developed as a by-product of our research and is not officially endorsed by the authors of the paper. 

:::

## Context

[ICML 2024](https://icml.cc/Conferences/2024) paper *Stop Making Unscientific AGI Performance Claims* [@altmeyer2024position] ([preprint](https://arxiv.org/abs/2402.03962), [blog post](https://www.patalt.org/blog/posts/eccco/), [code](https://github.com/pat-alt/spurious_sentience)):

::::{.columns}::::
:::{.column width='75%'}
- Even simple models can distill meaningful information that predicts external data.
- Humans are prone to seek patterns and anthropomorphize.
:::
:::{.column width='25%'}
![](www/qr.png)
:::
::::



## Basic Functionality

The package provides the following functionality:

- Load pre-processed data.
- Load the model proposed in the paper. 
- Basic model inference: compute forward passes and layer-wise activations.
- Download pre-computed activations for probing the model.

## Loading the Data {.smaller}

::::{.columns}::::
:::{.column width='50%'}

#### Sentences

40,000 time-stamped sentences from 

- meeting minutes
- press conferences
- speeches 

by members of the Federal Open Market Committee (FOMC):


```{julia}
#| output: true

using TrillionDollarWords
load_all_sentences() |>
  x -> names(x)
```


:::
:::{.column width='50%'}

#### All Data

Merged data includes economic indicators

- Consumer Price Index (CPI)
- Producer Price Index (PPI)
- US Treasury (UST) yields


```{julia}
#| output: true

load_all_data() |>
  x -> names(x)
```


:::
::::

## Loading the Model

- Can be loaded with or without the classifier head.
- Uses [Transformers.jl](https://github.com/chengchingwen/Transformers.jl) to retrieve the model from [HuggingFace](https://huggingface.co/gtfintechlab/FOMC-RoBERTa?text=A+very+hawkish+stance+excerted+by+the+doves).
-  Any keyword arguments accepted by `Transformers.HuggingFace.HGFConfig` can also be passed.


```{julia}
#| eval: false

load_model(; load_head=false, output_hidden_states=true)
```


## Basic Model Inference

::::{.columns}::::
:::{.column width='50%'}

#### From Scratch

Layer-wise activations can be computed as follows:


```{julia}
#| eval: false

df = load_all_sentences()
mod = load_model(
  load_head=false, 
  output_hidden_states=true
)
n = 5
queries = df[1:n, :]
layerwise_activations(
  mod, queries
) 
```

:::

:::{.column width='50%'}

#### From Artifacts

We have archived activations for each layer and sentence as [artifacts](https://github.com/pat-alt/TrillionDollarWords.jl/releases/tag/activations_2024-01-17):

``` julia
using LazyArtifacts

artifact"activations_layer_24"
```

> OK, but why would I need all this? 🤔

:::
::::


# "There! It's sentient!" {.nostretch} 

![](www/leo.png){fig-align="center"}



## Motivation {auto-animate=true}

- $A_1$: $enc($„It is essential to bring inflation back to target to avoid drifting into deflation territory.“$)$
- $A_2$: $enc($„It is essential to bring the numbers of doves back to target to avoid drifting into dovelation territory.“$)$

## Motivation {auto-animate=true .smaller}

::::{.columns}::::
:::{.column width='50%'}
- $A_1$: $enc($„It is essential to bring inflation back to target to avoid drifting into deflation territory.“$)$
- $A_2$: $enc($„It is essential to bring the numbers of doves back to target to avoid drifting into dovelation territory.“$)$

> "They're exactly the same."
> 
> --- Linear probe $\widehat{cpi}=f(A)$
:::
:::{.column width='50%'}
![](www/spider.jpeg)
:::
::::

## Embedding FOMC comms {.nostretch}

- We linearly probe all layers to predict unseen economic indicators (CPI, PPI, UST yields).
- Predictive power increases with layer depth and probes outperform simple AR($p$) models.

![Out-of-sample root mean squared error (RMSE) for the linear probe plotted against FOMC-RoBERTa’s n-th
layer for different indicators. ](www/mse_pca_128.png){#fig-mse width="80%"}  

## Sparks of Economic Understanding? {.nostretch}

If probe results were indicative of some intrinsic ‘understanding’, probe should not be sensitive to unrelated sentences.

![Probe predictions for sentences about inflation of prices (IP), deflation of prices (DP), inflation of birds (IB)
and deflation of birds (DB). The vertical axis shows predicted inflation levels subtracted by the average predicted value
of the probe for random noise.](www/attack_all_measures.png){#fig-attack width="80%"}

## Intended Purpose and Goals

Good starting point for the following ideas:

- Fine-tune additional models on the classification task or other tasks of interest.
- Further model probing, e.g. using other market indicators not discussed in the original paper.
- Improve and extend the label annotations. 

Any contributions are very much welcome.

# Questions? {.nostretch} 

With thanks to my co-authors Andrew M. Demetriou, Antony Bartlett, and Cynthia C. S. Liem and to the audience for their attention.

![](/www/images/qr.png){width="25%" fig-align="center"}

## References {.scrollable .smaller}

::: {#refs}
:::

## Image sources

- Leonardo DiCaprio: Meme template by user on <a href="https://www.reddit.com/r/MemeTemplatesOfficial/comments/g46e21/i_made_a_png_version_of_leonardo_dicaprio/#lightbox">Reddit</a>

## Quote sources

- "There! It's sentient"---that engineer at Google (probably!)
